In [2]:
# Step 1: Load the Data

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import re

Mounted at /content/drive


In [19]:
# Step 2: Inspect the Data

df_product = pd.read_csv('/content/drive/MyDrive/Data/Amazon_PPC_Files  /ProductReport.csv')

df_target = pd.read_csv('/content/drive/MyDrive/Data/Amazon_PPC_Files  /TargettingReport.csv')

import pandas as pd

def inspect_dataframe(df: pd.DataFrame,
                      name: str = "DataFrame",
                      key_col: str | None = None,
                      head_rows: int = 0):

    # Basic stats
    n_duplicates = df.duplicated().sum()

    print(f"=== {name} ===\n")
    print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns\n")
    print(f"Duplicate rows: {n_duplicates}")

    # Unique values in key column (if provided and exists)
    if key_col is not None and key_col in df.columns:
        print(f"Unique values in '{key_col}': {df[key_col].nunique()}")
    elif key_col is not None:
        print(f"Warning: key_col '{key_col}' not found in columns.")

    print(f"\nData types:\n{df.dtypes}")

    if head_rows > 0:
      print(f"\nFirst {head_rows} rows:\n{df.head(head_rows)}")

    missing = df.isnull().sum()
    if missing.any():
      print(f"\nMissing values (non-zero only):\n{missing[missing > 0]}")
    else:
      print("\nMissing values: None")

    print(f"\nNumeric summary (describe):\n{df.describe(include='number')}\n\n\n")

inspect_dataframe(df_product, "Product")
inspect_dataframe(df_target, "Target", None,2)

=== Product ===

Shape: 2683 rows × 25 columns

Duplicate rows: 0

Data types:
Date                                         object
Portfolio name                               object
Currency                                     object
Campaign Name                                object
Ad Group Name                                object
Retailer                                     object
Country                                      object
Advertised SKU                               object
Advertised ASIN                              object
Advertised Parent ASIN                       object
7 Day Total Sales                            object
Impressions                                   int64
Clicks                                        int64
Click-Thru Rate (CTR)                        object
Cost Per Click (CPC)                         object
Spend                                        object
Total Advertising Cost of Sales (ACOS)       object
Total Return on Advertising Spend (RO

In [20]:
# Step 3: Clean the Data

df_product['Date'] = pd.to_datetime(df_product['Date'])
df_target['Date'] = pd.to_datetime(df_target['Date'])


money_cols = [
    '7 Day Total Sales',
    'Spend',
    '7 Day Advertised SKU Sales',
    '7 Day Other SKU Sales'
]

for col in money_cols:
    df_product[col] = ( df_product[col].astype(str).str.replace(r'[\$,]', '', regex=True).str.strip().replace('', np.nan).astype(float) )
    df_target[col] = ( df_target[col].astype(str).str.replace(r'[\$,]', '', regex=True).str.strip().replace('', np.nan).astype(float) )



print(f"{df_product.dtypes.to_string()}\n\n{df_target.dtypes.to_string()}")

print("Total Spend (current df_target):", df_target['Spend'].sum())
print("Total Sales (current df_target):", df_target['7 Day Total Sales'].sum())





Date                                        datetime64[ns]
Portfolio name                                      object
Currency                                            object
Campaign Name                                       object
Ad Group Name                                       object
Retailer                                            object
Country                                             object
Advertised SKU                                      object
Advertised ASIN                                     object
Advertised Parent ASIN                              object
7 Day Total Sales                                  float64
Impressions                                          int64
Clicks                                               int64
Click-Thru Rate (CTR)                               object
Cost Per Click (CPC)                                object
Spend                                              float64
Total Advertising Cost of Sales (ACOS)              obje

In [21]:
# Step 4: Transform (add calculated fields)

# 1) CTR = Clicks / Impressions
df_product['ctr'] = np.where(
    df_product['Impressions'] > 0,
    df_product['Clicks'] / df_product['Impressions'],
    np.nan
)

# 2) Conversion Rate (CVR) = Orders / Clicks
df_product['cvr'] = np.where(
    df_product['Clicks'] > 0,
    df_product['7 Day Total Orders (#)'] / df_product['Clicks'],
    np.nan
)

# 3) CPC = Spend / Clicks
df_product['cpc'] = np.where(
    df_product['Clicks'] > 0,
    df_product['Spend'] / df_product['Clicks'],
    np.nan
)

# 4) ACOS = Spend / 7 Day Total Sales
df_product['acos'] = np.where(
    df_product['7 Day Total Sales'] > 0,
    df_product['Spend'] / df_product['7 Day Total Sales'],
    np.nan
)

# 5) ROAS = 7 Day Total Sales / Spend
df_product['roas'] = np.where(
    df_product['Spend'] > 0,
    df_product['7 Day Total Sales'] / df_product['Spend'],
    np.nan
)

# 6) Day of Week (for later analysis)
df_product['day_of_week'] = df_product['Date'].dt.day_name()


df_product[['Impressions', 'Clicks', 'Spend', '7 Day Total Sales',
        'ctr', 'cvr', 'cpc', 'acos', 'roas', 'day_of_week']].head()


,Impressions,Clicks,Spend,7 Day Total Sales,ctr,cvr,cpc,acos,roas,day_of_week
0,2,0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,Monday
1,4,0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,Tuesday
2,4,0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,Wednesday
3,7,0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,Thursday
4,8,1,0.6,0.0,0.125,0.0,0.6,NaN,0.0,Friday


In [22]:
# Step 5: Aggregate (group, pivot)

def safe_div(num, den):
    return np.where(den > 0, num / den, np.nan)

campaign_summary = (
    df_product.groupby('Campaign Name', as_index=False).agg(
        impressions=('Impressions', 'sum'),
        clicks=('Clicks', 'sum'),
        spend=('Spend', 'sum'),
        sales=('7 Day Total Sales', 'sum'),
        orders=('7 Day Total Orders (#)', 'sum'),
        units=('7 Day Total Units (#)', 'sum') )
)

# Derived metrics at campaign level
campaign_summary['ctr']  = safe_div(campaign_summary['clicks'], campaign_summary['impressions'])
campaign_summary['cvr']  = safe_div(campaign_summary['orders'], campaign_summary['clicks'])
campaign_summary['cpc']  = safe_div(campaign_summary['spend'],  campaign_summary['clicks'])
campaign_summary['acos'] = safe_div(campaign_summary['spend'],  campaign_summary['sales'])
campaign_summary['roas'] = safe_div(campaign_summary['sales'],  campaign_summary['spend'])

print("Campaign Pivot:\n")
display(campaign_summary)

asin_summary = (
   df_product.groupby('Advertised ASIN', as_index=False).agg(
        impressions=('Impressions', 'sum'),
        clicks=('Clicks', 'sum'),
        spend=('Spend', 'sum'),
        sales=('7 Day Total Sales', 'sum'),
        orders=('7 Day Total Orders (#)', 'sum'),
        units=('7 Day Total Units (#)', 'sum')  )
)

asin_summary['ctr']  = safe_div(asin_summary['clicks'], asin_summary['impressions'])
asin_summary['cvr']  = safe_div(asin_summary['orders'], asin_summary['clicks'])
asin_summary['cpc']  = safe_div(asin_summary['spend'],  asin_summary['clicks'])
asin_summary['acos'] = safe_div(asin_summary['spend'],  asin_summary['sales'])
asin_summary['roas'] = safe_div(asin_summary['sales'],  asin_summary['spend'])

print("\n\nASIN Pivot:\n")
display(asin_summary)


Campaign Pivot:



,Campaign Name,impressions,clicks,spend,sales,orders,units,ctr,cvr,cpc,acos,roas
0,DryTheRain- 8/16/2025 16:55:55.212,251,1,0.70,0.00,0,0,0.003984,0.000000,0.700000,NaN,0.000000
1,Eva - All Product - SP - Defense,4574,31,15.90,69.98,2,2,0.006777,0.064516,0.512903,0.227208,4.401258
2,Eva - Handle bar bag - B0CTTPN15W - SP - Auto,2588,40,15.97,0.00,0,0,0.015456,0.000000,0.399250,NaN,0.000000
3,Eva - SP - AI - B0CH4Z51T7 - PT - Performance ...,48853,731,401.84,1098.97,61,63,0.014963,0.083447,0.549713,0.365651,2.734845
4,Eva - SP - AI - B0CH4Z9LLW - B0DN1LY1L6 Loose ...,40303,358,193.83,205.28,12,12,0.008883,0.033520,0.541425,0.944223,1.059072
5,Eva - SP - AI - B0CW1X69SV - Loose Match - Aut...,18407,134,84.09,253.92,7,8,0.007280,0.052239,0.627537,0.331167,3.019622
6,Eva - SP - AI - B0D2Y39QRH - KT - frame bag -...,56838,605,422.22,727.01,19,19,0.010644,0.031405,0.697884,0.580762,1.721875
7,Eva - SP - All - KT:pack2ride - Phrase,20531,138,108.39,520.85,15,15,0.006722,0.108696,0.785435,0.208102,4.805333
8,Eva - SP - B0BKWYWHPH - KT - Exact - Converted,26,0,0.00,0.00,0,0,0.000000,NaN,NaN,NaN,NaN
9,Eva - SP - B0CH51YXMY - PT - Exact - Converted,271,6,2.53,0.00,0,0,0.022140,0.000000,0.421667,NaN,0.000000




ASIN Pivot:



,Advertised ASIN,impressions,clicks,spend,sales,orders,units,ctr,cvr,cpc,acos,roas
0,B0BKSZMNT9,598,5,2.72,34.99,1,1,0.008361,0.200000,0.544000,0.077736,12.863971
1,B0BKWYWHPH,68,1,0.08,0.00,0,0,0.014706,0.000000,0.080000,NaN,0.000000
2,B0BKX21KTL,10574,41,18.90,29.97,3,3,0.003877,0.073171,0.460976,0.630631,1.585714
3,B0BKX2XV7G,7132,68,15.56,9.99,1,1,0.009534,0.014706,0.228824,1.557558,0.642031
4,B0BL8HW6XM,323,2,1.13,0.00,0,0,0.006192,0.000000,0.565000,NaN,0.000000
5,B0BSC613Z7,5195,48,23.97,0.00,0,0,0.009240,0.000000,0.499375,NaN,0.000000
6,B0C5Y2JLZC,3338,11,8.33,34.99,1,1,0.003295,0.090909,0.757273,0.238068,4.200480
7,B0C6Y8G3G1,17924,97,54.79,242.93,6,7,0.005412,0.061856,0.564845,0.225538,4.433838
8,B0C6Y8LNFB,71267,820,378.12,2020.81,52,59,0.011506,0.063415,0.461122,0.187113,5.344362
9,B0C6Y9H8M5,15148,159,93.13,344.90,7,10,0.010496,0.044025,0.585723,0.270020,3.703425


In [23]:
Winners = (campaign_summary.sort_values(by='roas', ascending=False).head(5).copy())
losers = (campaign_summary[ campaign_summary['spend'] > 0 ].sort_values(by=['roas', 'spend'], ascending=[True, False]).head(5).copy())

display(losers)
display(Winners)



,Campaign Name,impressions,clicks,spend,sales,orders,units,ctr,cvr,cpc,acos,roas
2,Eva - Handle bar bag - B0CTTPN15W - SP - Auto,2588,40,15.97,0.0,0,0,0.015456,0.0,0.399250,NaN,0.0
14,Eva - SP - Inova - Auto - All,1905,23,12.80,0.0,0,0,0.012073,0.0,0.556522,NaN,0.0
16,Eva - SP - Mira - PT - Converted,603,9,3.06,0.0,0,0,0.014925,0.0,0.340000,NaN,0.0
9,Eva - SP - B0CH51YXMY - PT - Exact - Converted,271,6,2.53,0.0,0,0,0.022140,0.0,0.421667,NaN,0.0
17,Eva - SP - Toprock - PT - Converted - Exact,409,10,2.15,0.0,0,0,0.024450,0.0,0.215000,NaN,0.0


,Campaign Name,impressions,clicks,spend,sales,orders,units,ctr,cvr,cpc,acos,roas
12,Eva - SP - Bongo - B0C6Y8LNFB - Cat:Bike Handl...,24703,274,98.42,821.76,20,24,0.011092,0.072993,0.359197,0.119767,8.349522
7,Eva - SP - All - KT:pack2ride - Phrase,20531,138,108.39,520.85,15,15,0.006722,0.108696,0.785435,0.208102,4.805333
1,Eva - All Product - SP - Defense,4574,31,15.90,69.98,2,2,0.006777,0.064516,0.512903,0.227208,4.401258
11,Eva - SP - Bongo - Auto - All,75551,777,408.51,1751.89,44,51,0.010284,0.056628,0.525753,0.233182,4.288487
10,Eva - SP - B0CW1ZFDLK - Auto - Toprock,32142,341,148.35,625.81,19,19,0.010609,0.055718,0.435044,0.237053,4.218470


In [24]:
# Target_Type: ASIN / Category / Keyword
def get_target_type(x: str) -> str:
    s = str(x)
    if s.startswith('asin="'):
        return 'ASIN'
    elif s.startswith('category="'):
        return 'Category'
    else:
        return 'Keyword'

# Campaign_Type: Auto / Manual (match "Auto" or "AI" exactly, like REGEXMATCH("Auto|AI"))
def get_campaign_type(name: str) -> str:
    s = str(name)
    if re.search(r'Auto|AI', s):   # case-sensitive, no lower()
        return 'Auto'
    else:
        return 'Manual'

df_target['Target_Type']   = df_target['Targeting'].apply(get_target_type)
df_target['Campaign_Type'] = df_target['Campaign Name'].apply(get_campaign_type)

def safe_div(num, den):
    return np.where(den > 0, num / den, np.nan)

pivot_tc = (
    df_target
    .groupby(['Target_Type', 'Campaign_Type'], as_index=False)
    .agg(
        impressions=('Impressions', 'sum'),
        clicks=('Clicks', 'sum'),
        spend=('Spend', 'sum'),
        sales=('7 Day Total Sales', 'sum'),
        orders=('7 Day Total Orders (#)', 'sum')
    )
)

pivot_tc['ctr']  = safe_div(pivot_tc['clicks'], pivot_tc['impressions'])
pivot_tc['cvr']  = safe_div(pivot_tc['orders'], pivot_tc['clicks'])
pivot_tc['acos'] = safe_div(pivot_tc['spend'],  pivot_tc['sales'])
pivot_tc['roas'] = safe_div(pivot_tc['sales'],  pivot_tc['spend'])

print("\n\nTargeting Efficiency Pivot:\n")
display(pivot_tc)

def safe_div(num, den):
    return np.where(den > 0, num / den, np.nan)

# Pivot by Targeting
pivot_targeting = (
    df_target
    .groupby('Targeting', as_index=False)
    .agg(
        clicks=('Clicks', 'sum'),
        impressions=('Impressions', 'sum'),
        spend=('Spend', 'sum'),
        sales=('7 Day Total Sales', 'sum'),
        orders=('7 Day Total Orders (#)', 'sum'),
    )
)

# Derived metrics
pivot_targeting['ctr']  = safe_div(pivot_targeting['clicks'], pivot_targeting['impressions'])
pivot_targeting['cvr']  = safe_div(pivot_targeting['orders'], pivot_targeting['clicks'])
pivot_targeting['acos'] = safe_div(pivot_targeting['spend'],  pivot_targeting['sales'])
pivot_targeting['roas'] = safe_div(pivot_targeting['sales'],  pivot_targeting['spend'])

# Optional: sort by clicks or spend
pivot_targeting = pivot_targeting.sort_values('roas', ascending=True)

print("\n\nTargeting Pivot:\n")
display(pivot_targeting)




Targeting Efficiency Pivot:



,Target_Type,Campaign_Type,impressions,clicks,spend,sales,orders,ctr,cvr,acos,roas
0,ASIN,Auto,48853,731,401.84,1098.97,61,0.014963,0.083447,0.365651,2.734845
1,ASIN,Manual,7076,92,40.35,95.89,3,0.013002,0.032609,0.420795,2.376456
2,Category,Manual,24703,274,98.42,821.76,20,0.011092,0.072993,0.119767,8.349522
3,Keyword,Auto,56838,605,422.22,727.01,19,0.010644,0.031405,0.580762,1.721875
4,Keyword,Manual,20806,139,109.09,520.85,15,0.006681,0.107914,0.209446,4.774498




Targeting Pivot:



,Targeting,clicks,impressions,spend,sales,orders,ctr,cvr,acos,roas
0,"asin-expanded=""B00NT4KXBA""",1,226,0.70,0.00,0,0.004425,0.000000,NaN,0.000000
7,"asin=""B074XRWZH6""",6,219,2.16,0.00,0,0.027397,0.000000,NaN,0.000000
12,"asin=""B07RR1WX8Z""",3,366,0.90,0.00,0,0.008197,0.000000,NaN,0.000000
11,"asin=""B07H9KJ1W9""",4,108,1.62,0.00,0,0.037037,0.000000,NaN,0.000000
13,"asin=""B07YWXS4RL""",10,409,2.15,0.00,0,0.024450,0.000000,NaN,0.000000
15,"asin=""B097V45HQT""",3,173,1.35,0.00,0,0.017341,0.000000,NaN,0.000000
22,"asin=""B0CCS72KFF""",1,181,0.62,0.00,0,0.005525,0.000000,NaN,0.000000
18,"asin=""B0C5Y2JLZC""",8,418,4.32,0.00,0,0.019139,0.000000,NaN,0.000000
25,"asin=""B0CCYN34VT""",1,45,0.36,0.00,0,0.022222,0.000000,NaN,0.000000
28,"asin=""B0CXYSCWZY""",2,50,0.82,0.00,0,0.040000,0.000000,NaN,0.000000


In [28]:
negative = (pivot_targeting[pivot_targeting['spend'] > 0].sort_values(by=['roas', 'spend'], ascending=[True, False]).head(5).copy())


display(negative)
display(pivot_tc)

,Targeting,clicks,impressions,spend,sales,orders,ctr,cvr,acos,roas
18,"asin=""B0C5Y2JLZC""",8,418,4.32,0.0,0,0.019139,0.0,NaN,0.0
7,"asin=""B074XRWZH6""",6,219,2.16,0.0,0,0.027397,0.0,NaN,0.0
13,"asin=""B07YWXS4RL""",10,409,2.15,0.0,0,0.024450,0.0,NaN,0.0
11,"asin=""B07H9KJ1W9""",4,108,1.62,0.0,0,0.037037,0.0,NaN,0.0
24,"asin=""B0CCS95FRW""",3,841,1.56,0.0,0,0.003567,0.0,NaN,0.0


,Target_Type,Campaign_Type,impressions,clicks,spend,sales,orders,ctr,cvr,acos,roas
2,Category,Manual,24703,274,98.42,821.76,20,0.011092,0.072993,0.119767,8.349522
4,Keyword,Manual,20806,139,109.09,520.85,15,0.006681,0.107914,0.209446,4.774498
0,ASIN,Auto,48853,731,401.84,1098.97,61,0.014963,0.083447,0.365651,2.734845
1,ASIN,Manual,7076,92,40.35,95.89,3,0.013002,0.032609,0.420795,2.376456
3,Keyword,Auto,56838,605,422.22,727.01,19,0.010644,0.031405,0.580762,1.721875


In [30]:
!pip install xlsxwriter

import pandas as pd
import xlsxwriter

output_path = "/content/drive/MyDrive/GA/Amazon_ppc_dashboard.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:

    # 2) Executive summary sheet
    sheet_name = "EXECUTIVE_SUMMARY"
    Winners.to_excel(writer, sheet_name=sheet_name, startrow=3, index=False)

    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]
    heading_fmt = workbook.add_format({'bold': True, 'font_size': 14})

    # Winners heading at row 1
    worksheet.write(1, 0, "Winners (Campaigns / ASINs):", heading_fmt)

    # Compute rows dynamically based on Winners length
    winners_startrow = 3
    winners_nrows    = len(Winners) + 1  # +1 for header row

    # Losers section starts a few rows after Winners
    losers_title_row = winners_startrow + winners_nrows + 1   # blank row
    losers_startrow  = losers_title_row + 1

    worksheet.write(losers_title_row, 0, "Bleeders (Campaigns / ASINs):", heading_fmt)
    losers.to_excel(writer, sheet_name=sheet_name, startrow=losers_startrow, index=False)

    losers_nrows = len(losers) + 1

    # Targeting Efficiency section
    eff_title_row = losers_startrow + losers_nrows + 2   # gap of 2 rows
    eff_startrow  = eff_title_row + 1

    worksheet.write(eff_title_row, 0, "Targeting Efficiency by Type:", heading_fmt)
    pivot_tc.to_excel(writer, sheet_name=sheet_name, startrow=eff_startrow, index=False)

    eff_nrows = len(pivot_tc) + 1

    # Negative candidates section
    neg_title_row = eff_startrow + eff_nrows + 2
    neg_startrow  = neg_title_row + 1

    worksheet.write(neg_title_row, 0, "Negative Keyword / ASIN Candidates:", heading_fmt)
    negative.to_excel(writer, sheet_name=sheet_name, startrow=neg_startrow, index=False)
